## Setup

### Import libraries

In [ ]:
from arcgis.gis import GIS
from arcgis.graph import KnowledgeGraph
import pandas as pd
import os
import csv
import json
from datetime import datetime
from uuid import UUID

### Declare input parameters

In [ ]:
# output folder name
output_folder = r"C:\backups\myknowledgegraph_backup"

# Connecting to the GIS
gisPortalUrl = "https://adsrv2019.ad.local/portal"
username = "tkinlaw_ent"
password = "Esri.4.GIS"
url = r"https://adsrv2019.ad.local/server/rest/services/Hosted/HealthcareRecords/KnowledgeGraphServer" # provide the serice url for the input KG
gis = GIS(gisPortalUrl, username, password)

### Define folder and file names

In [ ]:
# output backup json file names
dm_ent = "datamodel_entities.json"
dm_rel = "datamodel_relationships.json"
all_ent = "all_entities.json"
all_rel = "all_relationships.json"
prov_file = "provenance_entities.json"  # this will only be used if you want to backup provenance records

## Create backup files

### Connect to portal and knowledge graph

In [ ]:
# connect to knowledge graph service
knowledgegraph_backup = KnowledgeGraph(url=url, gis=gis)

### Write data model entity types to backup json file

In [ ]:
# create list of formatted entity types
entity_types = []
for types in knowledgegraph_backup.datamodel["entity_types"]:
    curr_entity_type = {
        "name": knowledgegraph_backup.datamodel["entity_types"][types]["name"],
        "properties": knowledgegraph_backup.datamodel["entity_types"][types]["properties"]
        }
    entity_types.append(curr_entity_type)

In [ ]:
# write entity types to json file
with open(os.path.join(output_folder, dm_ent), "w") as f:
    json.dump(entity_types, f)

### Write data model relationship types to backup json file

In [ ]:
# create list of formatted relationship types
relationship_types = []
for types in knowledgegraph_backup.datamodel["relationship_types"]:
    curr_relationship_type = {
        "name": knowledgegraph_backup.datamodel["relationship_types"][types]["name"],
        "properties": knowledgegraph_backup.datamodel["relationship_types"][types]["properties"]
    }
    relationship_types.append(curr_relationship_type)

In [ ]:
# write relationship types to json file
with open(os.path.join(output_folder, dm_rel), "w") as f:
    json.dump(relationship_types, f)

### Write entities to backup json file

In [ ]:
# query for all entities in graph
original_entities = knowledgegraph_backup.query_streaming(
    "MATCH (n) RETURN distinct n"
)

In [ ]:
# create list of formatted entities to add to the graph
all_entities_fromquery = []
for entity in list(original_entities):
    curr_entity = entity[0]
    # convert UUID values to a string since json can't store UUIDs
    curr_entity["_id"] = str(curr_entity["_id"])
    for prop in curr_entity["_properties"]:
        if type(curr_entity["_properties"][prop]) == UUID:
            curr_entity["_properties"][prop] = str(curr_entity["_properties"][prop])
    # delete objectid, the server will handle creating new ones when we load the backup
    del curr_entity["_properties"][
        "objectid"
    ]
    all_entities_fromquery.append(curr_entity)

In [ ]:
# write entities list to json file
with open(os.path.join(output_folder, all_ent), "w") as f:
    json.dump(all_entities_fromquery, f)

### Write relationships to backup json file

In [ ]:
# query for all relationships in graph
original_relationships = knowledgegraph_backup.query_streaming(
    "MATCH ()-[rel]->() RETURN distinct rel"
)

In [ ]:
# create list of formatted entities to add to the graph
all_relationships_fromquery = []
for relationship in list(original_relationships):
    curr_relationship = relationship[0]
    # convert UUID values to a string since json can't store UUIDs
    curr_relationship["_id"] = str(curr_relationship["_id"])
    curr_relationship["_originEntityId"] = str(curr_relationship["_originEntityId"])
    curr_relationship["_destinationEntityId"] = str(
        curr_relationship["_destinationEntityId"]
    )
    for prop in curr_relationship["_properties"]:
        if type(curr_relationship["_properties"][prop]) == UUID:
            curr_relationship["_properties"][prop] = str(
                curr_relationship["_properties"][prop]
            )
    # delete objectid, the server will handle creating new ones when we load the backup
    del curr_relationship["_properties"][
        "objectid"
    ]
    all_relationships_fromquery.append(curr_relationship)

In [ ]:
# write relationships list to json file
with open(os.path.join(output_folder, all_rel), "w") as f:
    json.dump(all_relationships_fromquery, f)

## OPTIONAL: Write provenance records to backup json file

In [ ]:
# query for all provenance records in the graph
provenance_entities = knowledgegraph_backup.query_streaming(
    "MATCH (n:Provenance) RETURN distinct n", include_provenance=True
)

In [ ]:
# create list of formatted provenance records to the graph
all_provenance_fromquery = []
for entity in list(provenance_entities):
    curr_provenance = entity[0]
    # convert UUID values to a string since json can't store UUIDs
    curr_provenance["_id"] = str(curr_provenance["_id"])
    for prop in curr_provenance["_properties"]:
        if type(curr_provenance["_properties"][prop]) == UUID:
            curr_provenance["_properties"][prop] = str(
                curr_provenance["_properties"][prop]
            )
    # delete objectid, the server will handle creating new ones when we load the backup
    del curr_provenance["_properties"][
        "objectid"
    ]
    all_provenance_fromquery.append(curr_provenance)

In [ ]:
# write provenance list to json file
with open(os.path.join(output_folder, prov_file), "w") as f:
    json.dump(all_provenance_fromquery, f)

## Load backup files

### Connect to portal and create knowledge graph

In [ ]:
# create a knowledge graph without provenance enabled
result = gis.content.create_service(
    name="myknowledgegraph",
    capabilities="Query,Editing,Create,Update,Delete",
    service_type="KnowledgeGraph"
)
knowledgegraph_load = KnowledgeGraph(result.url, gis=gis)

The code above will need to be modified if we work with provenance in the class data

### Populate data model from saved json files


In [ ]:
# load data model json files into graph data model
with open(os.path.join(output_folder, dm_ent), "r") as file:
    dm_ents = json.load(file)
with open(os.path.join(output_folder, dm_rel), "r") as file:
    dm_rels = json.load(file)
knowledgegraph_load.named_object_type_adds(
    entity_types=dm_ents, relationship_types=dm_rels
)

### Get original documents names to correctly load data

In [ ]:
# get document entity type name
doc_type_name = "Document"
for entity_type in dm_ents:
    for prop in entity_type["properties"]:
        if entity_type["properties"][prop]["role"] == "esriGraphNamedObjectDocument":
            doc_type_name = entity_type["name"]

In [ ]:
# get document relationship type name
doc_rel_type_name = "HasDocument"
for relationship_type in dm_rels:
    for prop in relationship_type["properties"]:
        if (
            relationship_type["properties"][prop]["role"]
            == "esriGraphNamedObjectDocument"
        ):
            doc_rel_type_name = relationship_type["name"]

### Add additional document entity and relationship type properties

In [ ]:
# load any additional document entity type properties
origin_document_properties = None
for entity_type in dm_ents:
    if entity_type["name"] == doc_type_name:
        origin_document_properties = entity_type["properties"]
prop_list = []
for prop in origin_document_properties:
    prop_list.append(origin_document_properties[prop])
knowledgegraph_load.graph_property_adds(
    type_name="Document", graph_properties=prop_list
)

In [ ]:
# load any additional document relationship type properties
for relationship_type in dm_rels:
    if relationship_type["name"] == doc_rel_type_name:
        origin_document_rel_properties = relationship_type["properties"]
prop_list = []
for prop in origin_document_rel_properties:
    prop_list.append(origin_document_rel_properties[prop])
knowledgegraph_load.graph_property_adds(
    type_name="HasDocument", graph_properties=prop_list
)

### Get list of date properties from the data model json files

In [ ]:
date_properties = []
# add date property names for entity types
for types in dm_ents:
    for prop in types["properties"]:
        if types["properties"][prop]["fieldType"] == "esriFieldTypeDate":
            date_properties.append(prop)

In [ ]:
# add date property names for relationship types
for types in dm_rels:
    for prop in types["properties"]:
        if types["properties"][prop]["fieldType"] == "esriFieldTypeDate":
            date_properties.append(prop)

### Add all entities to the knowledge graph

In [ ]:
# load entities json file
with open(os.path.join(output_folder, all_ent), "r") as file:
    original_entities = json.load(file)
batch = []
for curr_entity in original_entities:
    # if a batch reaches 20k records, apply that batch of edits to the knowledge graph
    if len(batch) > 20000:
        result = knowledgegraph_load.apply_edits(adds=batch)
        batch = []
        # print error if one occurs during edit operation
        try:
            print(result["error"])
        except:
            print("No error adding entities")
    # in case original document type name is different, change name to Document
    if curr_entity["_typeName"] == doc_type_name:
        curr_entity["_typeName"] = "Document"
    # format UUID and date properties
    for prop in curr_entity["_properties"]:
        if prop in date_properties:
            try:
                curr_entity["_properties"][prop] = datetime.fromtimestamp(
                    int(curr_entity["_properties"][prop] / 1000)
                )
            except:
                curr_entity["_properties"][prop] = None
        try:
            curr_entity["_properties"][prop] = UUID(curr_entity["_properties"][prop])
        except:
            continue
    # format id UUID
    curr_entity["_id"] = UUID(curr_entity["_id"])
    batch.append(curr_entity)
# apply final batch of edits to the knowledge graph
result = knowledgegraph_load.apply_edits(adds=batch)
# print error if one occurs during edit operation
try:
    print(result["error"])
except:
    print("No error adding entities")

### Add all relationships to the knowledge graph

In [ ]:
# load relationships json file
with open(os.path.join(output_folder, all_rel), "r") as file:
    original_rels = json.load(file)
batch = []
for curr_relationship in original_rels:
    # if a batch reaches 20k records, apply that batch of edits to the knowledge graph
    if len(batch) > 20000:
        result = knowledgegraph_load.apply_edits(adds=batch)
        batch = []
        # print error if one occurs during edit operation
        try:
            print(result["error"])
        except:
            print("No error adding relationships")
    # in case original document type name is different, change name to HasDocument
    if curr_relationship["_typeName"] == doc_rel_type_name:
        curr_relationship["_typeName"] = "HasDocument"
    # format UUID and date properties
    for prop in curr_relationship["_properties"]:
        if prop in date_properties:
            try:
                curr_relationship["_properties"][prop] = datetime.fromtimestamp(
                    int(curr_relationship["_properties"][prop] / 1000)
                )
            except:
                curr_relationship["_properties"][prop] = None
        try:
            curr_relationship["_properties"][prop] = UUID(
                curr_relationship["_properties"][prop]
            )
        except:
            continue
    # format other relationship specific UUIDs
    curr_relationship["_id"] = UUID(curr_relationship["_id"])
    curr_relationship["_originEntityId"] = UUID(curr_relationship["_originEntityId"])
    curr_relationship["_destinationEntityId"] = UUID(
        curr_relationship["_destinationEntityId"]
    )
    batch.append(curr_relationship)
# apply final batch of edits to the knowledge graph
result = knowledgegraph_load.apply_edits(adds=batch)
# print error if one occurs during edit operation
try:
    print(result["error"])
except:
    print("No error adding relationships")

### Add search indexes to all text properties

In [ ]:
load_dm = knowledgegraph_load.datamodel
# add search indexes for all entity text properties
for entity_type in load_dm["entity_types"]:
    prop_list = []
    for prop in load_dm["entity_types"][entity_type]["properties"]:
        if (
            load_dm["entity_types"][entity_type]["properties"][prop]["fieldType"]
            == "esriFieldTypeString"
        ):
            prop_list.append(prop)
    knowledgegraph_load.update_search_index(
        adds={entity_type: {"property_names": prop_list}}
    )

In [ ]:
# add search indexes for all relationship text properties
for entity_type in load_dm["relationship_types"]:
    prop_list = []
    for prop in load_dm["relationship_types"][entity_type]["properties"]:
        if (
            load_dm["relationship_types"][entity_type]["properties"][prop]["fieldType"]
            == "esriFieldTypeString"
        ):
            prop_list.append(prop)
    knowledgegraph_load.update_search_index(
        adds={entity_type: {"property_names": prop_list}}
    )

### OPTIONAL: Add provenance records to the knowledge graph

# load provenance records json file
with open(os.path.join(output_folder, prov_file), "r") as file:
    prov_entities = json.load(file)

# add all provenance records
for curr_prov in prov_entities:
    # format UUID properties
    for prop in curr_prov["_properties"]:
        try:
            curr_prov["_properties"][prop] = UUID(curr_prov["_properties"][prop])
        except:
            continue
    # format id as UUID
    curr_prov["_id"] = UUID(curr_prov["_id"])
    # add provenance record
    knowledgegraph_load.apply_edits(adds=[curr_prov])